In [1]:
import argparse
import numpy as np
from itertools import count

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

In [89]:
import pandas as pd
import numpy as np

def getSamples(filename):
    data = pd.read_csv(filename, sep='\t')
    return data.values[:,1:].transpose()

data1 = getSamples("GSE2034-Normal-train.txt")
data2 = getSamples("GSE2034-Tumor-train.txt")

data1Label = np.zeros(len(data1)).reshape((-1, 1))
data2Label = np.ones(len(data2)).reshape((-1, 1))
x = np.concatenate((data1, data2))
y = np.concatenate((data1Label, data2Label))

In [90]:
dim = 12634
class Res1d(nn.Module):
    # the conv layers
    def __init__(self, inSize, outSize, kernel=(3,), strides=1,):
        super(Res1d, self).__init__()
        pding = (dim*strides + strides - dim + 2) // 2
        self.l1 = nn.Conv1d(inSize, outSize, kernel, stride=strides, padding=pding, bias=False)
        self.l2 = nn.InstanceNorm1d(outSize)

        if strides > 1 or inSize != outSize:
            if strides > 1:
                self.r1 = nn.ConstantPad1d((0,strides-1), 0)
                self.r2 = nn.AvgPool1d(strides)
            else:
                self.r1 = None
                self.r2 = None
            self.r3 = nn.Conv1d(inSize, outSize, 1, stride=strides, bias=False)
            self.r4 = nn.InstanceNorm1d(outSize)
    
        self.relu = nn.ReLU()
        
    def forward(self, x):
        l = self.l1(x)
        l = self.l2(l)
        
        if self.r1 is not None:
            r = self.r1(x)
            r = self.r2(r)
            print("r", r.shape)
            r = self.r3(r)
            r = self.r4(r)
        else:
            r = self.r3(x)
            r = self.r4(r)
            
        print(l.shape, r.shape)
        x = l + r
        return self.relu(x)
    
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(dim, 64)
        self.l2 = nn.ReLU()
        
        self.r1 = Res1d(1, 4, 3)
        
        self.r2 = Res1d(4, 8, 3)
        self.r3 = Res1d(8, 8, 3, strides=2)
        
        self.r4 = Res1d(8, 16, 3)
        self.r5 = Res1d(16, 16, 3, strides=2)
        
        self.r6 = Res1d(16, 32, 3)
        self.r7 = Res1d(32, 32, 3, strides=2)
        
        self.r8 = Res1d(32, 64, 3)
        self.r9 = Res1d(64, 64, 3, strides=2)
        
        self.r10 = Res1d(64, 128, 3)
        self.r11 = Res1d(128, 128, 3, strides=2)
        
        self.r12 = Res1d(128, 256, 3)
        self.r13 = Res1d(256, 256, 3, strides=2)
        
        self.r14 = Res1d(256, 512, 3)
        self.r15 = Res1d(512, 512, 3, strides=2)
        
        self.r16 = Res1d(512, 1024, 3)
        self.r17 = Res1d(1024, 1024, 3, strides=2)
        
        self.lastLinear = nn.Linear(1024+64,32)
        self.lastRelu = nn.ReLU()
        self.lastSigmoid = nn.Sigmoid()
    
    def forward(self, x):
        # shape is (batch, channel, time)
        l = x
        l = self.l2(self.l1(l))
        
        # conv layers should operate on time
        r = x
        r = self.r4(self.r3(self.r2(self.r1(r))))
        r = self.r8(self.r7(self.r6(self.r5(r))))
        r = self.r12(self.r11(self.r10(self.r9(r))))
        r = self.r16(self.r15(self.r14(self.r13(r))))
        r = self.r17(r)
        
        # flatten l
        l = l.view(x.shape[0],1, -1)
        
        y = torch.cat((l,r),0)
        y = self.lastLinear(y)
        y = self.lastRelu(y)
        y = self.lastSigmoid(y)
        return y

In [91]:
net = Net()

In [92]:
input = torch.randn(1, 1, dim)
net(input)

torch.Size([1, 4, 12634]) torch.Size([1, 4, 12634])
torch.Size([1, 8, 12634]) torch.Size([1, 8, 12634])
r torch.Size([1, 8, 6317])
torch.Size([1, 8, 12635]) torch.Size([1, 8, 3159])


RuntimeError: The size of tensor a (12635) must match the size of tensor b (3159) at non-singleton dimension 2